<a href="https://colab.research.google.com/github/PaulNjinu254/CNN1-Series-Fin/blob/main/CNN1_Series_Fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.datasets import mnist

# Conv1D Layer
class Conv1d:
    def __init__(self, n_filters, b_size, stride=1, padding=0):
        self.W = None
        self.b = None
        self.stride = stride
        self.pa = padding
        self.b_size = b_size
        self.n_filters = n_filters

    def init_weights(self, C, b_size):
        self.W = np.random.randn(self.n_filters, C, b_size) * 0.01
        self.b = np.zeros((self.n_filters,))

    def forward(self, x):
        self.x = np.pad(x, ((0, 0), (0, 0), (self.pa, self.pa)), mode='constant')
        N, C, L = self.x.shape
        out_length = (L - self.b_size) // self.stride + 1
        self.output_size = out_length

        col = np.zeros((N, self.W.shape[0], out_length))
        for i in range(out_length):
            col[:, :, i] = np.tensordot(
                self.x[:, :, i * self.stride:i * self.stride + self.b_size],
                self.W, axes=([1, 2], [1, 2])
            ) + self.b
        self.col = col
        return col

    def backward(self, d_out):
        N, C, L = self.x.shape
        dW = np.zeros_like(self.W)
        db = np.sum(d_out, axis=(0, 2))

        dx = np.zeros_like(self.x)
        for i in range((L - self.b_size) // self.stride + 1):
            window = self.x[:, :, i * self.stride:i * self.stride + self.b_size]
            for n in range(N):
                for f in range(self.n_filters):
                    dW[f] += d_out[n, f, i] * window[n]
                    dx[n, :, i * self.stride:i * self.stride + self.b_size] += d_out[n, f, i] * self.W[f]

        self.dW = dW
        self.db = db
        return dx[:, :, self.pa:-self.pa] if self.pa > 0 else dx

    def update(self, lr):
        self.W -= lr * self.dW
        self.b -= lr * self.db

# Fully Connected Layer
class FC:
    def __init__(self, in_dim, out_dim):
        self.W = np.random.randn(in_dim, out_dim) * 0.01
        self.b = np.zeros((1, out_dim))

    def forward(self, x):
        self.x = x
        return np.dot(x, self.W) + self.b

    def backward(self, d_out):
        self.dW = np.dot(self.x.T, d_out)
        self.db = np.sum(d_out, axis=0, keepdims=True)
        return np.dot(d_out, self.W.T)

    def update(self, lr):
        self.W -= lr * self.dW
        self.b -= lr * self.db

# Activation Functions
def relu(x):
    return np.maximum(0, x)

def relu_grad(x):
    return (x > 0).astype(float)

def softmax(x):
    exp = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp / np.sum(exp, axis=1, keepdims=True)

def cross_entropy(y_pred, y_true):
    return -np.sum(y_true * np.log(y_pred + 1e-9)) / y_pred.shape[0]

def delta_cross_entropy(y_pred, y_true):
    return (y_pred - y_true) / y_true.shape[0]

# CNN Classifier
class Scratch1dCNNClassifier:
    def __init__(self, num_epoch, lr, batch_size, n_features, n_nodes2, n_output, verbose=True):
        self.num_epoch = num_epoch
        self.lr = lr
        self.batch_size = batch_size
        self.verbose = verbose
        self.conv = Conv1d(n_filters=6, b_size=3, padding=1)
        self.conv.init_weights(1, 3)
        self.fc1 = FC(6 * n_features, n_nodes2)
        self.fc2 = FC(n_nodes2, n_output)

    def forward_propagation(self, X):
        self.z1 = self.conv.forward(X)
        self.a1 = relu(self.z1)
        self.a1_flat = self.a1.reshape(X.shape[0], -1)
        self.z2 = self.fc1.forward(self.a1_flat)
        self.a2 = relu(self.z2)
        self.z3 = self.fc2.forward(self.a2)
        self.a3 = softmax(self.z3)
        return self.a3

    def back_propagation(self, y):
        delta3 = delta_cross_entropy(self.a3, y)
        delta2 = self.fc2.backward(delta3)
        delta2 = delta2 * relu_grad(self.z2)
        delta1 = self.fc1.backward(delta2)
        delta1 = delta1.reshape(self.a1.shape)
        delta0 = delta1 * relu_grad(self.z1)
        self.conv.backward(delta0)
        self.fc2.update(self.lr)
        self.fc1.update(self.lr)
        self.conv.update(self.lr)

    def fit(self, X, y):
        for epoch in range(self.num_epoch):
            perm = np.random.permutation(X.shape[0])
            X_shuffled = X[perm]
            y_shuffled = y[perm]
            for i in range(0, X.shape[0], self.batch_size):
                X_batch = X_shuffled[i:i+self.batch_size]
                y_batch = y_shuffled[i:i+self.batch_size]
                self.forward_propagation(X_batch)
                self.back_propagation(y_batch)
            if self.verbose:
                y_pred = self.forward_propagation(X)
                loss = cross_entropy(y_pred, y)
                acc = self.accuracy(y_pred, y)
                print(f"Epoch {epoch+1} | Loss: {loss:.4f} | Accuracy: {acc:.4f}")

    def predict(self, X):
        y_pred = self.forward_propagation(X)
        return np.argmax(y_pred, axis=1)

    def accuracy(self, y_pred, y_true):
        return np.mean(np.argmax(y_pred, axis=1) == np.argmax(y_true, axis=1))

# Dummy Test
print("\nDummy Forward/Backward Test with small array")
dummy_X = np.random.rand(2, 1, 10)  # 2 samples, 1 channel, 10 length
dummy_y = np.array([[1, 0], [0, 1]])  # one-hot encoded

dummy_model = Scratch1dCNNClassifier(
    num_epoch=1, lr=0.01, batch_size=2,
    n_features=10, n_nodes2=5, n_output=2, verbose=False
)
dummy_model.fit(dummy_X, dummy_y)

Z3 = dummy_model.forward_propagation(dummy_X)
print("Forward output Z3 shape:", Z3.shape)

dummy_model.back_propagation(dummy_y)
print("\nBackward propagation successfully completed!\n")

# Estimate MNIST using Conv1D replacing part of FC

# Load MNIST and reshape properly (treat each row as a timestep)
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype(np.float32) / 255.0  # normalize
X_test = X_test.astype(np.float32) / 255.0

# Reshape for Conv1D: (batch, channels=1, length=28) treating each row (28 pixels) as a channel
X_train = X_train.transpose(0, 2, 1)  # shape becomes (N, 28, 28)
X_test = X_test.transpose(0, 2, 1)    # same

# Now reshape to (N, 1, 28*28) for compatibility with your Conv1d (uses (N, C, L))
X_train = X_train.reshape(-1, 1, 784)
X_test = X_test.reshape(-1, 1, 784)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_train_enc = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_enc = encoder.transform(y_test.reshape(-1, 1))

# Define improved model using Conv1d as feature extractor before FC
class ConvReplacesFC_MNIST:
    def __init__(self, num_epoch=5, lr=0.01, batch_size=128, kernel_size=5, n_filters=16, hidden_nodes=64, output_nodes=10, verbose=True):
        self.num_epoch = num_epoch
        self.lr = lr
        self.batch_size = batch_size
        self.verbose = verbose
        self.conv = Conv1d(n_filters=n_filters, b_size=kernel_size, padding=2)
        self.conv.init_weights(1, kernel_size)
        self.fc1 = FC(n_filters * 784, hidden_nodes)
        self.fc2 = FC(hidden_nodes, output_nodes)

    def forward_propagation(self, X):
        self.z1 = self.conv.forward(X)
        self.a1 = relu(self.z1)
        self.a1_flat = self.a1.reshape(X.shape[0], -1)
        self.z2 = self.fc1.forward(self.a1_flat)
        self.a2 = relu(self.z2)
        self.z3 = self.fc2.forward(self.a2)
        self.a3 = softmax(self.z3)
        return self.a3

    def back_propagation(self, y_true):
        delta3 = delta_cross_entropy(self.a3, y_true)
        delta2 = self.fc2.backward(delta3)
        delta2 *= relu_grad(self.z2)
        delta1 = self.fc1.backward(delta2)
        delta1 = delta1.reshape(self.a1.shape)
        delta0 = delta1 * relu_grad(self.z1)
        self.conv.backward(delta0)
        self.fc2.update(self.lr)
        self.fc1.update(self.lr)
        self.conv.update(self.lr)

    def fit(self, X, y):
        for epoch in range(self.num_epoch):
            perm = np.random.permutation(X.shape[0])
            X_shuffled = X[perm]
            y_shuffled = y[perm]
            for i in range(0, X.shape[0], self.batch_size):
                X_batch = X_shuffled[i:i+self.batch_size]
                y_batch = y_shuffled[i:i+self.batch_size]
                self.forward_propagation(X_batch)
                self.back_propagation(y_batch)
            if self.verbose:
                y_pred = self.forward_propagation(X)
                loss = cross_entropy(y_pred, y)
                acc = self.accuracy(y_pred, y)
                print(f"Epoch {epoch+1} | Loss: {loss:.4f} | Accuracy: {acc:.4f}")

    def accuracy(self, y_pred, y_true):
        return np.mean(np.argmax(y_pred, axis=1) == np.argmax(y_true, axis=1))

# Train on a subset for fast test
model = ConvReplacesFC_MNIST(num_epoch=5, lr=0.01, batch_size=128, verbose=True)
model.fit(X_train[:5000], y_train_enc[:5000])  # faster training on subset

# Evaluate on test data
y_test_pred = model.forward_propagation(X_test[:1000])
test_accuracy = model.accuracy(y_test_pred, y_test_enc[:1000])

print("\nEstimating MNIST with Conv1D replacing part of FC layer")
print(f"Accuracy: {test_accuracy:.4f}")


Dummy Forward/Backward Test with small array
Forward output Z3 shape: (2, 2)

Backward propagation successfully completed!

Epoch 1 | Loss: 2.3023 | Accuracy: 0.1172
Epoch 2 | Loss: 2.3021 | Accuracy: 0.1126
Epoch 3 | Loss: 2.3018 | Accuracy: 0.1126
Epoch 4 | Loss: 2.3016 | Accuracy: 0.1126
Epoch 5 | Loss: 2.3014 | Accuracy: 0.1276

Estimating MNIST with Conv1D replacing part of FC layer
Accuracy: 0.1420
